In [14]:
function read_federalistpapers()
    text = read("federalistpapers.txt", String)
    essays = split(text, r"FEDERALIST No.\ \d+")
    essays = essays[2:end-1] # remove the first and last elements
    println(essays)
    essays = map(x -> split(x, "To the People of the State of New York")[2], essays)

    essays = map(x -> split(x, "PUBLIUS")[1], essays)

    authors = ["Hamilton", "Madison", "Jay", "Unknown", "All"]
    author_list = []
    for essay in essays    

        if occursin("Hamilton", essay)
            push!(author_list, "Hamilton")
        elseif occursin("Madison", essay)
            push!(author_list, "Madison")
        elseif occursin("Jay", essay)
            push!(author_list, "Jay")
        elseif occursin("Unknown", essay)
            push!(author_list, "Unknown")
        end

    end
    
    
   
    #essays = map(x -> replace(x, r"\n"=>""), essays)
    essays = map(x -> split(x, "\r\n\r\n"), essays)
    
    return essays, author_list

end

function find_essays_by_author(essays, author)
    indices = findall(x -> x == author, author_list)
    return essays[indices]
end

function average_word_count(essays)
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        total_word_count += length(words)
    end
    return total_word_count/length(essays)
end

function average_word_length(essays)
    total_word_length = 0
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            total_word_length += length(word)
        end
        total_word_count += length(words)
    end
    return total_word_length/total_word_count
end

function average_sentence_count(essays)
    total_sentence_count = 0
    for essay in essays
        sentences = split(essay, r"[.?!]")
        total_sentence_count += length(sentences)
    end
    return total_sentence_count/length(essays)
end

function average_sentence_per_paragraph(essays)
    total_sentence_count = 0
    total_paragraph_count = 0
    for essay in essays
        paragraphs = split(essay, "\n")
        total_paragraph_count += length(paragraphs)
        for paragraph in paragraphs
            sentences = split(paragraph, r"[.?!]")
            total_sentence_count += length(sentences)
        end
    end
    return total_sentence_count/total_paragraph_count
end

function unique_word_count(essays)
    word_dict = Dict()
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            if haskey(word_dict, word)
                word_dict[word] += 1
            else
                word_dict[word] = 1
            end
        end
    end
    return word_dict
end

#essays,author_list = 
read_federalistpapers()

#=authors = ["Hamilton", "Madison", "Jay", "Unknown", "All"]

for author in authors
    author_essays = find_essays_by_author(essays, author)
    println(author, " wrote ", length(author_essays), " essays.")
    println("Average word count per essay: ", average_word_count(author_essays))
    println("Average word length per essay: ", average_word_length(author_essays))
    println("Average sentence count per essay: ", average_sentence_count(author_essays))
    println("Average sentence count per paragraph: ", average_sentence_per_paragraph(author_essays))
    word_dict = unique_word_count(author_essays)
    println("Unique words and their frequency:")
    for (word, count) in word_dict
        println(word, ": ", count)
    end
end
=#




SubString{String}["\ufeffThe Project Gutenberg eBook of The Federalist Papers, by Alexander Hamilton, John Jay, and James Madison\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: The Federalist Papers\r\n\r\nAuthor: Alexander Hamilton, John Jay, and James Madison\r\n\r\nRelease Date: July, 1998 [eBook #1404]\r\n[Most recently updated: December 20, 2021]\r\n\r\nLanguage: English\r\n\r\n\r\nProduced by: The Constitution Society, Anonymous Volunteers and David Widger\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE FEDERALIST PAPERS ***\r\n\r\n\r\n\r\n\r\nTHE F

In [ ]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

println(essays)

# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

array = editParagraphs(array)

# Question 9
authorSet = ["Hamilton", "Madison", "Jay", "Unknown", "All"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(paragraphsByAuthor[i]))")
end

#Incomplete
#=
function avgWordLength(authorEssays)
    wordCount = avgWordsPerEssay(authorEssays) * length(authorEssays)
end
=#

In [ ]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

println(essays)

# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

array = editParagraphs(array)

# Question 9
authorSet = ["Hamilton", "Madison", "Jay", "Unknown", "All"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(paragraphsByAuthor[i]))")
end

#Incomplete
#=
function avgWordLength(authorEssays)
    wordCount = avgWordsPerEssay(authorEssays) * length(authorEssays)
end
=#

In [ ]:
# Tester
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
println(length(essays))
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

# println(essays[1])

In [ ]:
# Minh's implementation

# Note: the number of the essay will be index no.

# Break it up into the individual essays. Each essay starts with "Federalist No. n." Within each essay you can skip everything up to and including "To the People of the State of New York:" and the footnotes after "PUBLIUS." You can also skip everything up to the start of the first essay and everything after the end of the last essay. (3 points)
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)

from = "To the People of the State of New York:"
to = "PUBLIUS" 

# Loop over each essay and extract the text in the essays
for (i, essay) in enumerate(essays)

    search_from = findfirst(from, essay)
    search_to = findfirst(to, essay)

    # Find the indices of from and to in the essay string
    start_index = search_from[1] + length(from)
    end_index = search_to[1] - 1
    
    # Extract the text between from and to
    extracted_text = essay[start_index:end_index]

    # Remove trailing new lines in the start and the end of the needed text
    extracted_text = rstrip(extracted_text) # somehow this does not remove the last line of the last paragraph, but it create a concrete pattern for all paragraphs (to have one new line at the end), so I guess it works.
    extracted_text = lstrip(extracted_text)

    # Print the extracted text
    # println(extracted_text)
    # println(i)

    # Replace with only the needed text from each essay
    essays[i] = extracted_text 
    
end
# println(typeof(essays[1]))
println(essays[1])

